In [120]:
import pandas as pd 

stock_data=pd.read_html("https://www.finanznachrichten.de/nachrichten/alle-empfehlungen.html")



In [121]:

len(stock_data)

4

In [122]:
stock_data_first=pd.DataFrame(stock_data[1])
print(stock_data_first.head())
stock_data_first.columns

    Zeit                               Aktuelle Nachrichten  \
0  19:22                GOLDMAN SACHS belässt WPP auf 'Buy'   
1  19:22          GOLDMAN SACHS belässt BHP GROUP auf 'Buy'   
2  18:46                  RBC belässt ASML auf 'Outperform'   
3  18:22        GOLDMAN SACHS belässt MERCK KGAA auf 'Sell'   
4  18:10  CREDIT SUISSE belässt MERCK KGAA auf 'Outperform'   

  Unternehmen / Aktien       PotentialKurse aktuell  Leser  Unnamed: 5  
0              WPP PLC   +14 % (9,20 GBP)9,080 Euro    425         NaN  
1        BHP GROUP PLC    +10 % (24 GBP)24,475 Euro    394         NaN  
2      ASML HOLDING NV  +34 % (610 Euro)455,30 Euro    472         NaN  
3           MERCK KGAA  -25 % (107 Euro)142,70 Euro    500         NaN  
4           MERCK KGAA  +16 % (165 Euro)142,70 Euro    490         NaN  


Index(['Zeit', 'Aktuelle Nachrichten', 'Unternehmen / Aktien',
       'PotentialKurse aktuell', 'Leser', 'Unnamed: 5'],
      dtype='object')

In [123]:
import re

stock_data_first['Aktuelle Nachrichten']=stock_data_first['Aktuelle Nachrichten'].str.findall('belässt\s([A-Z]*)').transform(''.join)
stock_data_first.head()

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5
0,19:22,WPP,WPP PLC,"+14 % (9,20 GBP)9,080 Euro",425,NaN
1,19:22,BHP,BHP GROUP PLC,"+10 % (24 GBP)24,475 Euro",394,NaN
2,18:46,ASML,ASML HOLDING NV,"+34 % (610 Euro)455,30 Euro",472,NaN
3,18:22,MERCK,MERCK KGAA,"-25 % (107 Euro)142,70 Euro",500,NaN
4,18:10,MERCK,MERCK KGAA,"+16 % (165 Euro)142,70 Euro",490,NaN


In [124]:
stock_data_first['Potential in %']=stock_data_first['PotentialKurse aktuell'].str.findall('([+-]\d*)\s%').transform(''.join).apply(pd.to_numeric)


In [55]:
stock_data_first.head()

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5,Potential in %
0,19:22,[WPP],WPP PLC,"+14 % (9,20 GBP)9,080 Euro",309,NaN,14.0
1,19:22,[BHP],BHP GROUP PLC,"+10 % (24 GBP)24,475 Euro",295,NaN,10.0
2,18:46,[ASML],ASML HOLDING NV,"+35 % (610 Euro)453,20 Euro",366,NaN,35.0
3,18:22,[MERCK],MERCK KGAA,"-25 % (107 Euro)142,70 Euro",411,NaN,-25.0
4,18:10,[MERCK],MERCK KGAA,"+16 % (165 Euro)142,70 Euro",394,NaN,16.0


In [125]:
stock_data_first=stock_data_first.sort_values(by=['Potential in %'],ascending=False)
stock_data_first.head()

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5,Potential in %
2,18:46,ASML,ASML HOLDING NV,"+34 % (610 Euro)455,30 Euro",472,NaN,34.0
17,15:22,C,CECONOMY AG,"+21 % (6,80 Euro)5,605 Euro",691,NaN,21.0
10,16:22,DWS,DWS GROUP GMBH & CO KGAA,"+20 % (42 Euro)35,115 Euro",624,NaN,20.0
16,15:22,DIC,DIC ASSET AG,"+17 % (16,50 Euro)14,100 Euro",695,NaN,17.0
4,18:10,MERCK,MERCK KGAA,"+16 % (165 Euro)142,70 Euro",490,NaN,16.0


In [126]:
from bs4 import BeautifulSoup
import requests

page=requests.get("https://www.finanznachrichten.de/nachrichten/alle-empfehlungen.html")
soup=BeautifulSoup(page.content,"html.parser")

In [127]:
best_five=stock_data_first.iloc[:5]

In [128]:
best_five

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5,Potential in %
2,18:46,ASML,ASML HOLDING NV,"+34 % (610 Euro)455,30 Euro",472,NaN,34.0
17,15:22,C,CECONOMY AG,"+21 % (6,80 Euro)5,605 Euro",691,NaN,21.0
10,16:22,DWS,DWS GROUP GMBH & CO KGAA,"+20 % (42 Euro)35,115 Euro",624,NaN,20.0
16,15:22,DIC,DIC ASSET AG,"+17 % (16,50 Euro)14,100 Euro",695,NaN,17.0
4,18:10,MERCK,MERCK KGAA,"+16 % (165 Euro)142,70 Euro",490,NaN,16.0


In [129]:
tables_list=[x for x in soup.find_all("table",attrs={"class":"rel-ct"})]

In [130]:
links=tables_list[0].tbody.find_all('a')
filtered_links=[x.get('href') for i,x in enumerate(links) if i%2==0]


In [160]:
best_links=list(map(lambda x:list([y for y in filtered_links if x in y]),best_five['Aktuelle Nachrichten'].str.lower()))
best_links=list(map(lambda x: x[0],best_links))
best_links

['https://www.finanznachrichten.de/nachrichten-2021-01/51784193-rbc-belaesst-asml-auf-outperform-322.htm',
 'https://www.finanznachrichten.de/nachrichten-2021-01/51784454-goldman-sachs-belaesst-wpp-auf-buy-322.htm',
 'https://www.finanznachrichten.de/nachrichten-2021-01/51782762-ubs-belaesst-dws-auf-buy-322.htm',
 'https://www.finanznachrichten.de/nachrichten-2021-01/51782025-baader-bank-belaesst-dic-asset-ag-auf-buy-322.htm',
 'https://www.finanznachrichten.de/nachrichten-2021-01/51784015-goldman-sachs-belaesst-merck-kgaa-auf-sell-322.htm']

In [161]:
isins=[]
for li in best_links:
    page_2=requests.get(li)
    soup_2=BeautifulSoup(page_2.content,"html.parser")
    span=soup_2.find('span',attrs={'class':"wkn-isin"})
    isins.append((span.find_all(id="produkt-isin")[0].get('data-isin'),li))

In [162]:
page_3=requests.get("https://www.onvista.de/aktien/-Aktie-JE00B8KF9B49")

[('NL0010273215',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51784193-rbc-belaesst-asml-auf-outperform-322.htm'),
 ('JE00B8KF9B49',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51784454-goldman-sachs-belaesst-wpp-auf-buy-322.htm'),
 ('DE000DWS1007',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51782762-ubs-belaesst-dws-auf-buy-322.htm'),
 ('DE000A1X3XX4',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51782025-baader-bank-belaesst-dic-asset-ag-auf-buy-322.htm'),
 ('DE0006599905',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51784015-goldman-sachs-belaesst-merck-kgaa-auf-sell-322.htm')]

(('JE00B8KF9B49',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51784454-goldman-sachs-belaesst-wpp-auf-buy-322.htm'),
 ('GB00BH0P3Z91',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51784455-goldman-sachs-belaesst-bhp-group-auf-buy-322.htm'),
 ('NL0010273215',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51784193-rbc-belaesst-asml-auf-outperform-322.htm'),
 ('DE0006599905',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51784015-goldman-sachs-belaesst-merck-kgaa-auf-sell-322.htm'),
 ('DE0006599905',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51783854-credit-suisse-belaesst-merck-kgaa-auf-outperform-322.htm'),
 ('US30231G1022',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51783624-credit-suisse-belaesst-exxon-mobil-corp-auf-neutral-322.htm'),
 ('DE000BASF111',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51783416-baader-bank-belaesst-basf-se-auf-buy-322.htm'),
 ('DE0006599905',
  'https://www.finanznachrichten.d